In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
#%config InlineBackend.figure_format = 'svg'
#%config InlineBackend.figure_format = 'pdf'

import kgof
import kgof.data as data
import kgof.density as density
import kgof.goftest as gof
import kgof.kernel as ker
import kgof.util as util
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as stats
import tensorflow as tf

In [2]:
# font options
font = {
    #'family' : 'normal',
    #'weight' : 'bold',
    'size'   : 18
}

plt.rc('font', **font)
plt.rc('lines', linewidth=2)
matplotlib.rcParams['pdf.fonttype'] = 42
matplotlib.rcParams['ps.fonttype'] = 42

## Shapiro-Wilk

In [63]:
# Can it work for the multivariate case?
#x = np.random.randn(200, 2).dot(np.random.rand(2, 2)) + 3
x = stats.t.rvs(df=5, size=500) 
st, pval = stats.shapiro(x)
print 'stat: {0}, pval: {0}'.format( st, pval)

stat: 0.990796685219, pval: 0.990796685219


## Anderson-Darling

In [66]:
result =  stats.anderson(x, dist='norm')
result

AndersonResult(statistic=1.0623991442279248, critical_values=array([ 0.571,  0.651,  0.781,  0.911,  1.083]), significance_level=array([ 15. ,  10. ,   5. ,   2.5,   1. ]))

In [68]:
A = np.reshape(range(12), (4, 3))
A

array([[ 0,  1,  2],
       [ 3,  4,  5],
       [ 6,  7,  8],
       [ 9, 10, 11]])

In [71]:
A*np.random.randn(3)

array([[-0.        ,  0.88595459, -0.63962194],
       [-2.95830589,  3.54381836, -1.59905484],
       [-5.91661179,  6.20168214, -2.55848774],
       [-8.87491768,  8.85954591, -3.51792064]])